In [2]:
!pip install yfinance
import pandas as pd, numpy as np
import yfinance as yf

     ---------------------------------------- 0.0/314.6 kB ? eta -:--:--
     - -------------------------------------- 10.2/314.6 kB ? eta -:--:--
     ------- ----------------------------- 61.4/314.6 kB 812.7 kB/s eta 0:00:01
     ----------------------- -------------- 194.6/314.6 kB 1.5 MB/s eta 0:00:01
     -------------------------------------- 314.6/314.6 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     - -------------------------------------- 0.1/3.0 MB 8.3 MB/s eta 0:00:01
     - -------------------------------------- 0.1/3.0 MB 8.3 MB/s eta 0:00:01
     -- ------------------------------------- 0.2/3.0 MB

In [3]:
#%% Heirarchical Indexing
"""
    Every index(including columns and index) in a DataFrame or a Series can have not only one layer.
"""
data = pd.Series(range(9),
                 index=[['a','a','a','b','b','c','c','d','d'],
                        ['x','y','z','x','y','x','y','x','y']])
data['a']
data['b']
data.loc[:,'x']

frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                     index=[['a','a','b','b'],
                            [1,2,1,2]],
                     columns=[['Ohio','Ohio','Colorado'],
                              ['Green','Red','Green']])
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']

frame['Ohio']

frame.loc[('a',1):('b',1),'Ohio']

idx = pd.IndexSlice
frame.loc[idx[:, 1], idx[:, "Red"]]

,state,Ohio
,color,Red
key1,key2,
a,1,1
b,1,7


In [4]:
#%% Stack / Unstack
"""
    Stack and Unstack can be used to reduce the dimention of the DataFrame
    When stacking, pandas will drop the nan by default, we can add "dropna=False" to avoid this.
    When unstcking, pandas will auto fill nan values.
"""
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                    index = ['Ohio', 'Colorado'],
                    columns = ['one', 'two', 'three'])
data.index.names = ['state']
data.columns.names = ['number']

data1=data.stack()
data1.unstack()
data1.unstack('state')
data1.unstack(0)
data1.unstack('number')
data1.unstack(1)

data = pd.DataFrame([[1,2,np.nan],
                     [3,np.nan,4]],
                    columns=['a','b','c'],
                    index=['one', 'two'])
data.stack()
data.stack(dropna=False)
data.stack().unstack()
data.stack().unstack(fill_value=-999)

frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                     index=[['a','a','b','b'],
                            [1,2,1,2]],
                     columns=[['Ohio','Ohio','Colorado'],
                              ['Green','Red','Green']])
frame.index.names = ['char', 'label']
frame.columns.names = ['state', 'color']
frame.unstack('char')
frame.unstack('label')
frame.T.unstack('state').T
frame.stack('state')
frame.T.unstack('color').T
frame.stack('state')

color                Green   Red
char label state                
a    1     Colorado      2   NaN
           Ohio          0   1.0
     2     Colorado      5   NaN
           Ohio          3   4.0
b    1     Colorado      8   NaN
           Ohio          6   7.0
     2     Colorado     11   NaN
           Ohio          9  10.0

In [78]:
#%% pivot / melt
"""
Similiar to stack and unstack, but can set more thing inside
"""

df = pd.DataFrame({'key':['foo', 'bar', 'baz'],
                   'A': [1,2,3],
                   'B': [4,5,6],
                   'C': [7,8,9]})
print(df)

melted = pd.melt(df)
print(melted)

melted_2 = pd.melt(df, ['key'])
print(melted_2)

   key  A  B  C
0  foo  1  4  7
1  bar  2  5  8
2  baz  3  6  9
   variable value
0       key   foo
1       key   bar
2       key   baz
3         A     1
4         A     2
5         A     3
6         B     4
7         B     5
8         B     6
9         C     7
10        C     8
11        C     9
   key variable  value
0  foo        A      1
1  bar        A      2
2  baz        A      3
3  foo        B      4
4  bar        B      5
5  baz        B      6
6  foo        C      7
7  bar        C      8
8  baz        C      9


In [88]:
melted_2 = pd.melt(df, ['key'])
reshaped_pivot = melted_2.pivot(columns='variable', index='key', values='value')
melted_2.set_index(['key', 'variable'], inplace=True)
reshaped_unstack = melted_2.unstack('variable').T.reset_index(level=0,drop=True).T

print(reshaped_pivot)
print(reshaped_unstack)

variable  A  B  C
key              
bar       2  5  8
baz       3  6  9
foo       1  4  7
variable  A  B  C
key              
bar       2  5  8
baz       3  6  9
foo       1  4  7


In [90]:
df = pd.DataFrame({'key':['foo', 'bar', 'baz'],
                   'A': [1,2,3],
                   'B': [4,5,6],
                   'C': [7,8,9]})
melted = pd.melt(df)
melted_2 = pd.melt(df, ['key'])
melted_2['value_2'] = range(9,0,-1)

reshaped = melted_2.pivot(columns='variable', index='key', values='value')
reshaped = melted_2.pivot(columns='variable', index='key', values='value_2')
reshaped = melted_2.pivot(columns='variable', index='key')

In [91]:
#%% group by
"""
    split-apply-combine
   
Step 1. Grouping(splitting)
    We need to provide "key", the cretiria for grouping, to the goupby function.
    The key can be:
        1. list, array (must be the same length)
        2. columns in the DataFrame
        3. dictionary or series
        4. function
ps: 
    we can use for loop to iterate the grouped result
"""
df = pd.DataFrame({'key1':['a', 'a', 'b', 'b', 'a'],
                   'key2':['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)},
                  index=range(1,6))

In [92]:
# groupby list
grouped_lis = df.groupby([1,1,2,2,3])
for key, group in grouped_lis:
    print(key)
    print(group)
    print()

1
  key1 key2     data1     data2
1    a  one -0.112945 -0.089901
2    a  two -0.238803  0.568040

2
  key1 key2     data1     data2
3    b  one  0.713748  0.657477
4    b  two -0.105042 -0.397958

3
  key1 key2     data1     data2
5    a  one -1.148591  0.173943



In [93]:
# groupby conlumn
grouped_col = df.groupby('key1')
groups = {}
for key, group in grouped_col:
    print('key: ', key)
    print('data: \n', group)
    groups[key]=group
    print('------------------------')
grouped_col_1 = df.groupby('key1', as_index=False)
for key, group in grouped_col_1:
    print('key: ', key)
    print('data: \n', group)
    print('------------------------')
grouped_col_2 = df.groupby('key1', as_index=True)
for key, group in grouped_col_2:
    print('key: ', key)
    print('data: \n', group)
    print('------------------------')
grouped_col2 = df.groupby(['key1','key2'])
groups = {}
for key, group in grouped_col2:
    print('key: ', key)
    print('data: \n', group)
    groups[key]=group
    print('------------------------')

key:  a
data: 
   key1 key2     data1     data2
1    a  one -0.112945 -0.089901
2    a  two -0.238803  0.568040
5    a  one -1.148591  0.173943
------------------------
key:  b
data: 
   key1 key2     data1     data2
3    b  one  0.713748  0.657477
4    b  two -0.105042 -0.397958
------------------------
key:  a
data: 
   key1 key2     data1     data2
1    a  one -0.112945 -0.089901
2    a  two -0.238803  0.568040
5    a  one -1.148591  0.173943
------------------------
key:  b
data: 
   key1 key2     data1     data2
3    b  one  0.713748  0.657477
4    b  two -0.105042 -0.397958
------------------------
key:  a
data: 
   key1 key2     data1     data2
1    a  one -0.112945 -0.089901
2    a  two -0.238803  0.568040
5    a  one -1.148591  0.173943
------------------------
key:  b
data: 
   key1 key2     data1     data2
3    b  one  0.713748  0.657477
4    b  two -0.105042 -0.397958
------------------------
key:  ('a', 'one')
data: 
   key1 key2     data1     data2
1    a  one -0.112945 -

In [94]:
# groupby dictionary (use index or columns)
people = pd.DataFrame(np.random.randn(5,5),
                      columns=['a','b','c','d','e'],
                      index=['Joe', 'Steve', 'Wes', 'Jane', 'Julia'])
dic = {'a':'red',
       'b':'red',
       'c':'blue',
       'd':'blue',
       'e':'red',
       'f':'yellow'}
grouped_dic = people.groupby(dic, axis=1)

dic_2 = {'Joe': 'g1', 
         'Steve': 'g1', 
         'Wes': 'g2', 
         'Jane': 'g2', 
         'Julia': 'g3'}
grouped_dic_2 = people.groupby(dic_2, axis=0)

C:\Users\yhn12\AppData\Local\Temp\ipykernel_41928\2224169845.py:11: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped_dic = people.groupby(dic, axis=1)
C:\Users\yhn12\AppData\Local\Temp\ipykernel_41928\2224169845.py:18: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  grouped_dic_2 = people.groupby(dic_2, axis=0)


In [95]:
## groupby series
ser = pd.Series(dic)
grouped_ser = people.groupby(ser, axis=1)

ser_2 = pd.Series(dic_2)
grouped_ser_2 = people.groupby(ser_2, axis=0)

C:\Users\yhn12\AppData\Local\Temp\ipykernel_41928\277045232.py:3: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped_ser = people.groupby(ser, axis=1)
C:\Users\yhn12\AppData\Local\Temp\ipykernel_41928\277045232.py:6: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  grouped_ser_2 = people.groupby(ser_2, axis=0)


In [96]:
# groupby function (use index or columns as inputs of the function)
grouped_fun = people.groupby(len, axis=1)
grouped_fun_2 = people.groupby(len, axis=0)
grouped_fun_title = people.groupby(lambda x: x[0], axis=1)
grouped_fun_title_2 = people.groupby(lambda x: x[0], axis=0)

C:\Users\yhn12\AppData\Local\Temp\ipykernel_41928\4013220265.py:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped_fun = people.groupby(len, axis=1)
C:\Users\yhn12\AppData\Local\Temp\ipykernel_41928\4013220265.py:3: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  grouped_fun_2 = people.groupby(len, axis=0)
C:\Users\yhn12\AppData\Local\Temp\ipykernel_41928\4013220265.py:4: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped_fun_title = people.groupby(lambda x: x[0], axis=1)
C:\Users\yhn12\AppData\Local\Temp\ipykernel_41928\4013220265.py:5: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  grouped_fun_title_2 = people.groupby(lambda x: x[0], axis=0)


In [102]:
"""  
Step 2. Applying method on every group, and get the result
    We can use some functions already optimized:
        1. count()
        2. sum()
        3. mean()
        4. median()
        5. std(), var()
        6. min(), max()
        7. prod()
        8. fist(), last()
    or all other series or dataframe function:
        1. descripe()
    or any user-defined function, using "apply" and "aggregate"
    We can even apply serval methods on one columns, apply different method on different column, or both.
"""
df = pd.DataFrame({'key1':['a', 'a', 'b', 'b', 'a', 'a', 'a', 'b', 'b', 'a'],
                   'key2':['one', 'two', 'one', 'two', 'one', 'one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(10),
                   'data2': np.random.randn(10)},
                  index=range(1,11))
grouped_key1 = df.groupby('key1')

In [103]:
df

,key1,key2,data1,data2
1,a,one,-0.195134,-0.353686
2,a,two,-0.484692,-0.071595
3,b,one,0.128055,-0.284606
4,b,two,1.037671,0.751929
5,a,one,-0.519199,-1.050450
6,a,one,-0.663597,0.148084
7,a,two,-0.903274,0.289209
8,b,one,1.850976,0.139995
9,b,two,1.971298,-1.534207
10,a,one,0.953924,-0.665480


In [105]:
## Using built-in functions
count_group = df.groupby('key1').count()
sum_group = df.groupby('key1').sum()
mean_group = df.groupby('key1').mean(numeric_only=True)
prod_group = df.groupby('key1').prod(numeric_only=True)
first_group = df.groupby('key1').first()
last_group = df.groupby('key1').last()

descr_group = df.groupby('key1').describe()

In [107]:
## Using user-defined functions: groupby.apply()
"""
When using "groupby.apply(fun)", the input of the function must be a dataframe. 
Pandas will automatically concate the result dataframe from different groups into a big final dataframe.
"""
df = pd.DataFrame({'key1':['a', 'a', 'b', 'b', 'a', 'a', 'a', 'b', 'b', 'a'],
                   'key2':['one', 'two', 'one', 'two', 'one', 'one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(10),
                   'data2': np.random.randn(10)},
                  index=range(1,11))
def top_3(df, n=3, sort_col='data1'):
    return df.sort_values(by=sort_col)[-n:]
def top_n(a, n, sort_col):
    return a.sort_values(by=sort_col)[-n:]

appy_top_3 = df.groupby('key1').apply(top_3)
appy_top_3_noindex = df.groupby('key1').apply(top_3)
appy_top_4 = df.groupby('key1').apply(top_n, n=4, sort_col='data2')

In [108]:
## (Optional) two arguments in groupby: as_index & group_keys
df = pd.DataFrame({'key1':['a', 'a', 'b', 'b', 'a', 'a', 'a', 'b', 'b', 'a'],
                   'key2':['one', 'two', 'one', 'two', 'one', 'one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(10),
                   'data2': np.random.randn(10)},
                  index=range(1,11))
top3_ori = df.groupby('key1').apply(top_3)
top3_AsIndexFalse = df.groupby('key1', as_index=False).apply(top_3)
top3_GroupKeysFalse = df.groupby('key1', group_keys=False).apply(top_3)

In [109]:
## Using user-defined functions: groupby.aggregate(agg)
"""
When using "groupby.agg(fun)", the input of the function must be a list-like object. 
Work just like DataFrame.agg()
Pandas will automatically concate the result dataframe from different groups into a big final dataframe.
"""

'\nWhen using "groupby.agg(fun)", the input of the function must be a list-like object. \nWork just like DataFrame.agg()\nPandas will automatically concate the result dataframe from different groups into a big final dataframe.\n'

In [114]:
## (Supplementary Material) DataFrame.agg()
"""
"DataFrame.agg()" is a enhanced eddition of "DataFrame.apply()". 
When using apply(), we can only apply one function column-by-column or row-by-row, 
    but when using agg(), we can apply serveral funcitons to each column or apply specific functions to specific columns.
"""
df = pd.DataFrame({'data1': np.random.randn(10),
                   'data2': np.random.randn(10)},
                  index=range(1,11))
df_agg_single = df.agg('mean')
df_agg_multi_1 = df.agg(['mean','sum'])
df_agg_multi_1 = df.agg(['mean', 'sum'])
df_agg_mulit_2 = df.agg({'data1': ['mean','sum','std'],
                         'data2': ['min', 'max']})

## groupby.agg()
df = pd.DataFrame({'key1':['a', 'a', 'b', 'b', 'a', 'a', 'a', 'b', 'b', 'a'],
                   'data1': np.random.randn(10),
                   'data2': np.random.randn(10)},
                  index=range(1,11))
grouped_agg_0 = df.groupby('key1').agg('mean')
grouped_agg_multi_1 = df.groupby('key1').agg(['mean','sum'])
grouped_agg_multi_1_rename = df.groupby('key1').agg([('group_mean', 'mean'),
                                                     ('group_sum', 'sum')])
grouped_agg_multi_2 = df.groupby('key1').agg({'data1': ['mean','sum','std'],
                                              'data2': ['min', 'max']})

def max_min_avg(ser):
    return (max(ser)-min(ser))/ser.mean()

group_agg_multi_3 = df.groupby('key1').agg({'data1': ['mean','sum','std'],
                                            'data2': ['min', 'max', max_min_avg]})
group_agg_multi_rename = df.groupby('key1').agg({'data1': [('group_mean', 'mean'),
                                                           ('group_sum', 'sum'),
                                                           ('group_std', 'std')],
                                                 'data2': ['min', 'max', max_min_avg]})